# Text to Fashion Images

## 1. Upload data to S3
Here I use pokeman dataset as an example, which is composed of 833 image-text pairs. To scale up, you can just process your data into the same format.

In [1]:
import sagemaker
import boto3
import datetime
import json
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [14]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

prefix_train = 'datasets/heguan_reformed'
inputs_train = 's3://sagemaker-us-west-2-452145973879/datasets/heguan_reformed/'

# local_data_path = "example_data"
# local_data_path = "../data/heguan-reformed"
# inputs_train = sagemaker_session.upload_data(path=local_data_path, key_prefix=prefix_train)
# print(inputs_train)

# prefix_test = 'datasets/brushnet_test'
# inputs_train = sagemaker_session.upload_data(path = "test_data", key_prefix=prefix_train)
# print(inputs_train)

## 2. Upload checkpoints to S3

In [15]:
# prefix_base_model = 'checkpoints/realisticVisionV60B1_v51VAE'
# inputs_base_model = sagemaker_session.upload_data(path="ckpt/realisticVisionV60B1_v51VAE", key_prefix=prefix_base_model)
# print(inputs_base_model)

In [16]:
# prefix_brushnet_model = 'checkpoints/random_mask_brushnet_ckpt'
# inputs_brushnet_model = sagemaker_session.upload_data(path="ckpt/random_mask_brushnet_ckpt", key_prefix=prefix_brushnet_model)
# print(inputs_brushnet_model)

## 2. Start a training job

In [17]:
import time
from sagemaker.estimator import Estimator

region = sagemaker_session.boto_session.region_name

image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker'

instance_count = 1
# instance_type = 'ml.g5.12xlarge'
instance_type = 'ml.p4d.24xlarge' ## p4d - 8*40G / p4de - 8*80G
# instance_type = 'ml.g5.48xlarge'

#     'TRAIN_DIR': '/opt/ml/input/data/train'
environment = {
    'NODE_NUMBER': str(instance_count),
    'TRAIN_DATA_PATH': f's3://{bucket}/{prefix_train}/',
#    'TEST_DATA_PATH': f's3://{bucket}/{prefix_test}/',
    'BASE_MODEL_S3_PATH': f's3://{bucket}/checkpoints/realisticVisionV60B1_v51VAE/',
    'BRUSHNET_MODEL_S3_PATH': f's3://{bucket}/checkpoints/random_mask_brushnet_ckpt/', 
    'OUTPUT_MODEL_S3_PATH': f's3://{bucket}/checkpoints/BrushNet_{instance_type}_v2/', # destination
#     'OUTPUT_DIR': '/opt/ml/model'
}

estimator = Estimator(role=role,
                      entry_point='entry.py',
                      source_dir='./sm_scripts',
                      base_job_name='brushnet-launch',
                      instance_count=instance_count,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      max_run=2*24*3600, #任务最大存续时间，默认2day，需要提交ticket提升quota最大28天
                      disable_profiler=True,
                      debugger_hook_config=False)

estimator.fit({'train': inputs_train})

INFO:sagemaker:Creating training-job with name: brushnet-launch-2024-04-17-01-51-03-631


2024-04-17 01:52:55 Starting - Starting the training job...
2024-04-17 01:53:17 Pending - Training job waiting for capacity...
2024-04-17 01:53:42 Pending - Preparing the instances for training...........................
2024-04-17 01:58:02 Downloading - Downloading input data.........
2024-04-17 01:59:52 Downloading - Downloading the training image...
2024-04-17 02:00:18 Training - Training image download completed. Training in progress......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-17 02:01:11,904 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-17 02:01:12,004 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-17 02:01:12,013 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-17 02:01:12,015 sagemaker_pytorch_container.training INFO     Invoking user training script.

KeyboardInterrupt: 